In [6]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 16px;
    font-family: Monaco;
}

div.output_area pre {
    font-size: 12px;
}
</style>

In [2]:
import os
os.chdir(os.getcwd()+"./..")

In [3]:
from project_modules.utils import get_squad_v2_pandas_dataframe

In [4]:
squad_train_df, squad_dev_df = get_squad_v2_pandas_dataframe()

In [5]:
squad_train_df.head()

,id,title,context,question,answer_text,answer_start,is_impossible
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269,False
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207,False
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526,False
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166,False
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276,False


In [7]:
squad_model_path = r"E:\MyFiles\WorkSpace\BertModels\QNLI\albert-base-v2-fine-tuned-squad-sample"

In [8]:
import torch

In [9]:
from transformers import AutoTokenizer
from transformers import AlbertForQuestionAnswering

In [10]:
class model_config():
    model_path = r"E:\MyFiles\WorkSpace\BertModels\SQUAD2\albert-base-v2-fine-tuned-squad-sample"
    max_length = 512
    batch_size = 12

In [11]:
from transformers import AlbertForQuestionAnswering

In [12]:
device = device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
model_config.model_path

'E:\\MyFiles\\WorkSpace\\BertModels\\SQUAD2\\albert-base-v2-fine-tuned-squad-sample'

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_config.model_path)

In [15]:
from project_modules.datasets import TextPairsDataset

In [16]:
example_id = 10

In [17]:
question_list = squad_train_df['question'].iloc[:example_id].to_list()
context_list = squad_train_df['context'].iloc[:example_id].to_list()

In [18]:
text_pairs_datasets = TextPairsDataset(question_list,context_list,tokenizer,512,device)

In [19]:
from  torch.utils.data import DataLoader

In [20]:
text_pairs_datasets_loader = DataLoader(text_pairs_datasets)

In [21]:
model_config.model_path

'E:\\MyFiles\\WorkSpace\\BertModels\\SQUAD2\\albert-base-v2-fine-tuned-squad-sample'

In [22]:
model  = AlbertForQuestionAnswering.from_pretrained(model_config.model_path)

In [23]:
model = model.to(device)

In [24]:
model = model.eval()

In [25]:
token_inputs  = text_pairs_datasets[:4]

In [26]:
with torch.no_grad():
    output= model(**token_inputs)

In [27]:
start_logits = output.start_logits.cpu().detach().numpy()
end_logits = output.end_logits.cpu().detach().numpy()

In [28]:
input_ids = token_inputs['input_ids']
result_dict_list = []

In [29]:
max_answer_len = 32

In [30]:
for idx in range(len(start_logits)):
    result_dict = {}
    start_end = (0,0)
    start_end_score = (-1,-1)

    score = -1

    for start,p_start in enumerate(start_logits[idx]):
        if p_start>0:
            for end,p_end in enumerate(end_logits[idx]):
                if p_end>0:
                    if end >= start and end < start + max_answer_len:
                        if p_start * p_end > score:
                            start_end = (start,end)
                            start_end_score = (p_start,p_end)
                            score = p_start * p_end
    start,end = start_end
    start_score,end_score = start_end_score

    pred_answer = ""
    if start!=0 and end !=0:
        pred_answer = tokenizer.decode(
            input_ids[idx][start:end+1]
        )

    result_dict['start_pos'] = start
    result_dict['start_score'] = start_score
    result_dict['end_pos'] = end
    result_dict['end_score'] = end_score

    result_dict['answer'] = pred_answer
    result_dict['score'] = score

    result_dict_list.append(result_dict)

In [31]:
result_dict_list

[{'start_pos': 79,
  'start_score': 5.986222,
  'end_pos': 81,
  'end_score': 6.6467805,
  'answer': 'late 1990s',
  'score': 39.7891},
 {'start_pos': 70,
  'start_score': 6.395449,
  'end_pos': 73,
  'end_score': 6.4965677,
  'answer': 'singing and dancing competitions',
  'score': 41.54847},
 {'start_pos': 86,
  'start_score': 5.5977654,
  'end_pos': 88,
  'end_score': 6.1736636,
  'answer': 'late 1990s',
  'score': 34.55872},
 {'start_pos': 60,
  'start_score': 7.1698217,
  'end_pos': 62,
  'end_score': 7.2510285,
  'answer': 'houston, texas',
  'score': 51.988583}]